In [1]:
import sys
import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Bar

設定cluster csv路徑並讀取資料

In [2]:
cluster_path = './cluster-result/1713836452_som/cluster_130.csv'
cluster = pd.read_csv(cluster_path)

根據cluster進行劃分

In [3]:
from src.describe import summary_cluster

chart = summary_cluster(cluster)
chart.render_notebook()